# Sentiment Analysis Agent Demo

This notebook demonstrates the Sentiment Analysis Agent's capabilities:
- Resolving company names to ticker symbols
- Retrieving recent news articles from MongoDB
- Analyzing FinBERT sentiment scores
- Interpreting market sentiment and investor mood

The agent uses Llama 3.1 8B running locally via Ollama.

In [1]:
# Cell 1: Imports
import sys
sys.path.insert(0, '..')  # If running from notebooks folder

from evaluation.phoenix_tracer import launch_phoenix, setup_tracing

# Cell 2: Launch Phoenix
session = launch_phoenix()
# This opens Phoenix UI in a new browser tab

# Cell 3: Setup tracing
setup_tracing()

c:\Users\Ryan Chin\Documents\UM CS\WIH3001 Data Science Project\RoundtableAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Python312\Lib\contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based index ix_cumulative_llm_token_count_total
  next(self.gen)
C:\Python312\Lib\contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based index ix_latency
  next(self.gen)


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix
Phoenix launched at: http://localhost:6006/
OpenTelemetry Tracing Details
|  Phoenix Project: roundtable-ai
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.

Tracing enabled for project: roundtable-ai


True

## 1. Setup and Imports

In [1]:
# Add project root to path
import sys
sys.path.insert(0, '..')

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Import the Sentiment Agent
from agents import SentimentAgent, create_sentiment_agent, get_llm

print("Imports successful!")

Imports successful!


## 2. Initialize the LLM (Gemini)

Connect to Gemini-2.0-flash.


In [4]:
# Connect to Ollama LLM
llm = get_llm(
    model_name="gemini-2.0-flash",
    temperature=0.3,
)

Initializing Gemini model: gemini-2.0-flash
Successfully connected to Gemini model: gemini-2.0-flash


## 3. Create the Sentiment Agent

In [5]:
# Create the agent with the loaded LLM
sentiment_agent = create_sentiment_agent(llm=llm)

# Display agent metadata
print("Agent Type:", sentiment_agent.agent_type)
print("Description:", sentiment_agent.agent_description)
print("\nCapabilities:")
for cap in sentiment_agent.get_capabilities():
    print(f"  - {cap}")

Agent Type: sentiment
Description: Analyzes news articles and FinBERT sentiment scores to gauge market perception and investor sentiment

Capabilities:
  - News article retrieval
  - FinBERT sentiment analysis
  - Sentiment trend identification
  - Market mood assessment
  - News event impact analysis
  - Sentiment-based risk identification


## 4. Test: Ticker Resolution

Test if the agent can resolve company names to ticker symbols.

In [6]:
# Test ticker resolution
response = sentiment_agent.chat(
    "What is the ticker symbol for Genting Berhad?",
    thread_id="test-ticker"
)
print(response)

[values] {'messages': [HumanMessage(content='What is the ticker symbol for Genting Berhad?', additional_kwargs={}, response_metadata={}, id='636a37b3-fd58-425f-b797-74683002ffd6')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'resolve_ticker_symbol', 'arguments': '{"company_name": "Genting Berhad"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--019b2811-3c3b-7ef1-9d81-404ce7be898b-0', tool_calls=[{'name': 'resolve_ticker_symbol', 'args': {'company_name': 'Genting Berhad'}, 'id': '815682ec-a4f7-4453-85cd-0005fc015322', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1307, 'output_tokens': 12, 'total_tokens': 1319, 'input_token_details': {'cache_read': 0}})]}}
[values] {'messages': [HumanMessage(content='What is the ticker symbol for Ge

## 5. Test: Article Retrieval

Test if the agent can retrieve recent news articles.

In [6]:
# Test article retrieval
response = sentiment_agent.chat(
    "Get the recent news articles for Maybank (1155.KL) from the past 7 days. How many articles are there?",
    thread_id="test-articles"
)
print(response)

[values] {'messages': [HumanMessage(content='Get the recent news articles for Maybank (1155.KL) from the past 7 days. How many articles are there?', additional_kwargs={}, response_metadata={}, id='abee469d-ea4a-4e76-b1e7-757f61d6c703')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_recent_articles', 'arguments': '{"ticker": "1155", "days": 7}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--019b171f-5273-7f83-97d7-035b57f96abb-0', tool_calls=[{'name': 'get_recent_articles', 'args': {'ticker': '1155', 'days': 7}, 'id': 'b2bd8649-67d6-41b4-83b0-bdccfaffcea6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1304, 'output_tokens': 12, 'total_tokens': 1316, 'input_token_details': {'cache_read': 0}})]}}
[values] {'messages': [HumanMessage

In [7]:
# Get more details about the articles
response = sentiment_agent.chat(
    "What are the headlines of these articles? Summarize the main topics being discussed.",
    thread_id="test-articles"
)
print(response)

[values] {'messages': [HumanMessage(content='Get the recent news articles for Maybank (1155.KL) from the past 7 days. How many articles are there?', additional_kwargs={}, response_metadata={}, id='abee469d-ea4a-4e76-b1e7-757f61d6c703'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_recent_articles', 'arguments': '{"ticker": "1155", "days": 7}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--019b171f-5273-7f83-97d7-035b57f96abb-0', tool_calls=[{'name': 'get_recent_articles', 'args': {'ticker': '1155', 'days': 7}, 'id': 'b2bd8649-67d6-41b4-83b0-bdccfaffcea6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1304, 'output_tokens': 12, 'total_tokens': 1316, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='{\'success\': True, \'ticker\': \'1155\', \'lookbac

## 6. Test: Sentiment Score Analysis

Test if the agent can retrieve and interpret FinBERT sentiment scores.

In [ ]:
# Get sentiment scores
response = sentiment_agent.chat(
    "What are the sentiment scores for recent Maybank articles? Are they mostly positive, negative, or neutral?",
    thread_id="test-sentiment"
)
print(response)

[values] {'messages': [HumanMessage(content='What are the sentiment scores for recent Maybank articles? Are they mostly positive, negative, or neutral?', additional_kwargs={}, response_metadata={}, id='d3bebc88-3ebb-4d84-9ace-0aa9a01631d1')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'resolve_ticker_symbol', 'arguments': '{"company_name": "Maybank"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--019b16a3-1302-74b0-b115-9ad4295e26c5-0', tool_calls=[{'name': 'resolve_ticker_symbol', 'args': {'company_name': 'Maybank'}, 'id': 'b4f075fe-e9ad-4a70-8b92-c2f0f6bb7c79', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1681, 'output_tokens': 10, 'total_tokens': 1691, 'input_token_details': {'cache_read': 0}})]}}
[values] {'messages': [HumanMe

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: MAYBANK"}}}


[updates] {'tools': {'messages': [ToolMessage(content='{"success": false, "query": "Maybank", "error": "Could not resolve \'Maybank\' to ticker symbol.", "suggestions": ["Try using the stock ticker directly (e.g. 1155.KL for Maybank)", "Check spelling of the company name", "Use the full official company name like \'Malayan Banking Berhad\'"]}', name='resolve_ticker_symbol', id='2b02ef23-7731-42e5-928e-0809e7bcad82', tool_call_id='b4f075fe-e9ad-4a70-8b92-c2f0f6bb7c79')]}}
[values] {'messages': [HumanMessage(content='What are the sentiment scores for recent Maybank articles? Are they mostly positive, negative, or neutral?', additional_kwargs={}, response_metadata={}, id='d3bebc88-3ebb-4d84-9ace-0aa9a01631d1'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'resolve_ticker_symbol', 'arguments': '{"company_name": "Maybank"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'saf

In [9]:
# Get sentiment scores
response = sentiment_agent.chat(
    "Yes",
    thread_id="test-sentiment"
)
print(response)

[values] {'messages': [HumanMessage(content='What are the sentiment scores for recent Maybank articles? Are they mostly positive, negative, or neutral?', additional_kwargs={}, response_metadata={}, id='d3bebc88-3ebb-4d84-9ace-0aa9a01631d1'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'resolve_ticker_symbol', 'arguments': '{"company_name": "Maybank"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--019b16a3-1302-74b0-b115-9ad4295e26c5-0', tool_calls=[{'name': 'resolve_ticker_symbol', 'args': {'company_name': 'Maybank'}, 'id': 'b4f075fe-e9ad-4a70-8b92-c2f0f6bb7c79', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1681, 'output_tokens': 10, 'total_tokens': 1691, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='{"success": false, "query": "Maybank", "error"

## 7. Test: Comprehensive Sentiment Analysis

Test the full analysis using the `analyze_sentiment` method.

In [8]:
# Comprehensive sentiment analysis using convenience method
analysis = sentiment_agent.analyze_sentiment(
    company="Malayan Banking Berhad",
    thread_id="analysis-maybank"
)
print(analysis)

[values] {'messages': [HumanMessage(content='Please analyze the market sentiment for Malayan Banking Berhad over the past 7 days.\n\nInclude:\n1. Company identification\n2. Summary of recent news articles and headlines\n3. Sentiment distribution (positive/negative/neutral percentages)\n4. Key themes or events driving sentiment\n5. Sentiment trend assessment (improving, stable, or deteriorating)\n6. Implications for investors based on current sentiment', additional_kwargs={}, response_metadata={}, id='a95b81b6-b668-41ab-819d-b28819e47062')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'resolve_ticker_symbol', 'arguments': '{"company_name": "Malayan Banking Berhad"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--019b171f-d5a5-7712-842a-cb15f30

## 8. Test: Follow-up Questions (Conversation Memory)

Test that the agent maintains conversation history.

In [9]:
# Start a conversation
thread_id = "conversation-test"

response1 = sentiment_agent.chat(
    "Analyze the market sentiment for CIMB Group.",
    thread_id=thread_id
)
print("Initial Analysis:")
print(response1)
print("\n" + "="*80 + "\n")

[values] {'messages': [HumanMessage(content='Analyze the market sentiment for CIMB Group.', additional_kwargs={}, response_metadata={}, id='2f9e53fb-eb6b-4d8e-82ec-79c3ad985c7b')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'resolve_ticker_symbol', 'arguments': '{"company_name": "CIMB Group"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--019b1720-50d6-7a53-91bb-7914a576c07f-0', tool_calls=[{'name': 'resolve_ticker_symbol', 'args': {'company_name': 'CIMB Group'}, 'id': '9a74b51b-2f05-42a1-abc7-e5c867461381', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1284, 'output_tokens': 12, 'total_tokens': 1296, 'input_token_details': {'cache_read': 0}})]}}
[values] {'messages': [HumanMessage(content='Analyze the market sentiment for CIMB Gro

In [10]:
# Follow-up question (should remember context)
response2 = sentiment_agent.chat(
    "Are there any negative news articles I should be concerned about?",
    thread_id=thread_id
)
print("Follow-up (Negative News):")
print(response2)

[values] {'messages': [HumanMessage(content='Analyze the market sentiment for CIMB Group.', additional_kwargs={}, response_metadata={}, id='2f9e53fb-eb6b-4d8e-82ec-79c3ad985c7b'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'resolve_ticker_symbol', 'arguments': '{"company_name": "CIMB Group"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--019b1720-50d6-7a53-91bb-7914a576c07f-0', tool_calls=[{'name': 'resolve_ticker_symbol', 'args': {'company_name': 'CIMB Group'}, 'id': '9a74b51b-2f05-42a1-abc7-e5c867461381', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1284, 'output_tokens': 12, 'total_tokens': 1296, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='{"success": true, "query": "CIMB Group", "ticker": "1023.KL", "company_name": "CIMB GROUP HOLDINGS BER

In [11]:
# Another follow-up
response3 = sentiment_agent.chat(
    "Based on the sentiment analysis, is the market outlook bullish or bearish for this stock?",
    thread_id=thread_id
)
print("Follow-up (Market Outlook):")
print(response3)

[values] {'messages': [HumanMessage(content='Analyze the market sentiment for CIMB Group.', additional_kwargs={}, response_metadata={}, id='2f9e53fb-eb6b-4d8e-82ec-79c3ad985c7b'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'resolve_ticker_symbol', 'arguments': '{"company_name": "CIMB Group"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--019b1720-50d6-7a53-91bb-7914a576c07f-0', tool_calls=[{'name': 'resolve_ticker_symbol', 'args': {'company_name': 'CIMB Group'}, 'id': '9a74b51b-2f05-42a1-abc7-e5c867461381', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1284, 'output_tokens': 12, 'total_tokens': 1296, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='{"success": true, "query": "CIMB Group", "ticker": "1023.KL", "company_name": "CIMB GROUP HOLDINGS BER

## 9. Test: Different Time Windows

In [ ]:
# Short-term sentiment (3 days)
response = sentiment_agent.chat(
    "What is the sentiment trend for Tenaga Nasional over the past 3 days? Any recent news events?",
    thread_id="short-term"
)
print("Short-term Sentiment (3 days):")
print(response)

In [ ]:
# Longer-term sentiment (14 days)
response = sentiment_agent.chat(
    "Analyze the sentiment evolution for Tenaga Nasional over the past 14 days. Has sentiment been improving or deteriorating?",
    thread_id="long-term"
)
print("Longer-term Sentiment (14 days):")
print(response)

## 10. Test: Specific Sentiment Questions

In [ ]:
# Ask about sentiment distribution
response = sentiment_agent.chat(
    """For Public Bank, analyze recent news sentiment and provide:
    1. Percentage breakdown (positive/negative/neutral)
    2. Most significant positive news
    3. Most significant negative news (if any)
    4. Overall sentiment assessment""",
    thread_id="sentiment-breakdown"
)
print(response)

In [12]:
# Ask about sentiment impact
response = sentiment_agent.chat(
    "How might the current market sentiment for Petronas Chemicals affect its stock price in the near term?",
    thread_id="sentiment-impact"
)
print(response)

[values] {'messages': [HumanMessage(content='How might the current market sentiment for Petronas Chemicals affect its stock price in the near term?', additional_kwargs={}, response_metadata={}, id='a4e93e27-4478-4d70-9e4b-528a627c6c67')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'resolve_ticker_symbol', 'arguments': '{"company_name": "Petronas Chemicals"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--019b1721-905b-7ce0-81ae-6e0150bd7a5b-0', tool_calls=[{'name': 'resolve_ticker_symbol', 'args': {'company_name': 'Petronas Chemicals'}, 'id': 'f9637b5d-05bd-4211-b02b-6bfa648ddd32', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1294, 'output_tokens': 12, 'total_tokens': 1306, 'input_token_details': {'cache_read': 0}})]}}
[values] {'m

## 11. Metadata for Multi-Agent Orchestration

Test the methods designed for future multi-agent debate.

In [ ]:
# Get agent metadata (for orchestration)
metadata = sentiment_agent.get_agent_metadata()
print("Agent Metadata:")
print(metadata)

In [ ]:
# Format response for debate
debate_response = sentiment_agent.format_response_for_debate(
    response="Market sentiment is predominantly positive with 70% favorable coverage. I recommend a bullish stance.",
    confidence=0.65
)
print("Formatted for Debate:")
print(debate_response)

## Summary

This notebook demonstrated:
- ✅ Loading Llama 3.1 8B locally with 4-bit quantization
- ✅ Creating a Sentiment Analysis Agent
- ✅ Resolving company names to ticker symbols
- ✅ Retrieving news articles from MongoDB
- ✅ Analyzing FinBERT sentiment scores
- ✅ Interpreting sentiment trends and distributions
- ✅ Maintaining conversation history across messages
- ✅ Preparing for multi-agent orchestration